# Imports

In [1]:
import jax
import jax.numpy as jnp 
from jax import vmap, grad, jit
import optax
from sample_trajectory import *
import numpy as np

# Globals

In [2]:
horizon = 2000

# Sampling trajectories

In [20]:
np.random.wald(1, 1**2)

0.4934553951929208

In [29]:
sample_saccade(jax.random.PRNGKey(0), -1)

AttributeError: `np.issctype` was removed in the NumPy 2.0 release. Use `issubclass(rep, np.generic)` instead.

In [28]:
a = tfp.distributions.InverseGaussian(loc=1., concentration=1.)
a.sample(sample_shape=(1,1), seed=jax.random.PRNGKey(0))

OverflowError: Python int too large to convert to C long

In [6]:
jnp.concat([jnp.arange(4), jnp.arange(3)])

Array([0, 1, 2, 3, 0, 1, 2], dtype=int32)

In [11]:
def power(x, s = 2):
    return x**s

def square_power(y, f):
    return f(y, s=3)

In [12]:
square_power(3, power)

27